# Word Ladder Game

**Getting Dictionary**

First we'll get a publically available english dictionary. We can use any other language dictionary as well.

In [4]:
import urllib3
http = urllib3.PoolManager()
r = http.request('GET',"https://docs.oracle.com/javase/tutorial/collections/interfaces/examples/dictionary.txt", preload_content = False)
with open("MyWord.txt", 'wb') as out:
    data = r.read()
    out.write(data)
r.release_conn()
word_list = open("MyWord.txt").readlines()
word_list = list(map(str.strip, word_list))
len(word_list)

80368

## Data Cleaning

In this step we'll make a word list of all the words have alphabet in it and with word legnth equal to 3 (we can use words with word length greater than 3 also)

In [5]:
word_list = [word for word in word_list if len(word) == 3]
word_list = [word for word in word_list if word.isalpha()]
word_list = list(map(str.lower, word_list))
len(word_list)

972

In [6]:
import numpy as np
word_list = np.asarray(word_list)
word_list.sort()
word_list.dtype

dtype('<U3')

## Data Compression

In [8]:
word_bytes = np.ndarray((word_list.size, word_list.itemsize), dtype = 'uint8', buffer = word_list.data)
print(word_bytes.shape)
word_bytes = word_bytes[:, ::word_list.itemsize//3]
print(word_bytes.shape)

(972, 12)
(972, 3)


**Calculation of hamming distance**

In [18]:
print(word_list[:4])
print(word_bytes[:4])
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
hamming_dist = pdist(word_bytes, metric= 'hamming')
print(hamming_dist)
print(hamming_dist.shape)

['aah' 'aal' 'aas' 'aba']
[[ 97  97 104]
 [ 97  97 108]
 [ 97  97 115]
 [ 97  98  97]]
[0.33333333 0.33333333 0.66666667 ... 0.66666667 0.66666667 0.33333333]
(471906,)


In [12]:
print(squareform(hamming_dist)[0:4, 0:4])
mat = squareform(hamming_dist < 1.5/3)
print(mat.shape)
print(mat[0:4, 0:4])
graph = csr_matrix(mat)

[[0.         0.33333333 0.33333333 0.66666667]
 [0.33333333 0.         0.33333333 0.66666667]
 [0.33333333 0.33333333 0.         0.66666667]
 [0.66666667 0.66666667 0.66666667 0.        ]]
(972, 972)
[[False  True  True False]
 [ True False  True False]
 [ True  True False False]
 [False False False False]]


**Example**

Taking two words 'and' and 'dog'. Let's check the hamming distance between them and print the ladder.

In [14]:
i1 = word_list.searchsorted('and')
i2 = word_list.searchsorted('dog')
print(word_list[i1])
print(word_list[i2])

and
dog


In [15]:
from scipy.sparse.csgraph import dijkstra
distances, predecessors = dijkstra(graph, indices = i1, return_predecessors = True)
print(distances[i2])

4.0


In [17]:
path = []
i = i2
while i != i1:
    path.append(word_list[i])
    i = predecessors[i]
path.append(word_list[i1])
print(path[::-1])

['and', 'aid', 'did', 'dig', 'dog']
